## Overview

This repository contains an implementation of the 2D Hasegawa-Wakatani equations using Nektar++. The equations are stated as

\begin{align*}
  \frac{\partial\zeta}{\partial t} + [\phi, \zeta] &= \alpha (\phi - n) \\
  \frac{\partial n}{\partial t} + [\phi, n] &= \alpha (\phi - n) - \kappa \frac{\partial\phi}{\partial y}
\end{align*}

where $\zeta$ is the vorticity, $n$ is the perturbed number density, $\phi$ is the electrostatic potential, and

$$[a,b] = \frac{\partial a}{\partial x} \frac{\partial b}{\partial y} - \frac{\partial a}{\partial y} \frac{\partial b}{\partial x}$$

is the canonical Poisson bracket operator. The vorticity and electrostatic potential are related through the Poisson equation $\nabla^2\phi = \zeta$. $\alpha$ is the adiabiacity operator (taken to be constant in this solver), and $\kappa$ is the background density gradient scale length.

## Numerical implementation

In the enclosed solver, the equations are solved in a similar manner [as the approach outlined by Ammar Hakim](http://ammar-hakim.org/sj/je/je17/je17-hasegawa-wakatani.html). We formulate the above equations in conservative form as

$$ \frac{\partial \mathbf{u}}{\partial t} + \nabla \cdot \mathbf{F}(\mathbf{u}) = \mathbf{G}(\mathbf{u}) $$

where $\mathbf{u} = (\zeta, n)$, $\mathbf{F}(\mathbf{u}) = [ \mathbf{v}_E \phi, \mathbf{v}_E \zeta ]$ with the drift velocity $\mathbf{v}_E = (\partial_y \phi, -\partial_x \phi)$, and $\mathbf{G}$ contains the remaining source terms on the right hand side. This is then discretised using a discontinuous Galerkin formulation; this has the advantage of increasingly stability and means that the hyperviscosity term that sometimes appears on the right hand side is not required.

Further details on implementation:

- timestepping is performed explicitly: Nektar++ supports a number of explicit timestepping methods through the general linear method for timestepping (see Vos et al, Int. J. Comp. Fluid Dyn. 25 (2011), pp 107-125 for details and the Nektar++ user guide for a list of supported timestepping methods).
- at each timestep, the electrostatic potential is solved however in a continuous Galerkin (CG) setting, which is more optimized for the Poisson solve.

## Example simulation

The `example` directory contains a setup for this case with a simple square, meshed using a $64\times64$ quadrilateral grid at order 3, with $\kappa = 1$, $\alpha = 2$ and initialised with a Gaussian field so that

$$ n(\mathbf{x}) = \phi(\mathbf{x}) = \exp(-\|\mathbf{x}\|^2/s^2), \quad \zeta(\mathbf{x}) = 4\exp(-\|\mathbf{x}\|^2/s^2)\frac{\|\mathbf{x}\|^2 - s^2}{s^4} $$

with $s=2$. Simulation setup (e.g. parameters, boundary conditions) is stored in the `driftwave.xml` session file, with the grid in the `square_quads.xml` session. To run the solver in serial and assuming you are running inside the `docker` environment, run
```
cd example
DriftWaveSolver driftwave.xml square_quads.xml
```
or in parallel, run
```
cd example
mpirun -n $NPROCS DriftWaveSolver driftwave.xml square_quads.xml
```